In [1]:
import numpy as np
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
from sklearn import metrics
import pickle as pk
import pandas as pd
from sklearn.model_selection import train_test_split
import time

In [32]:
NN_arch = pk.load(open('NN_arch.pk', 'rb'))
test=pd.read_csv('test.csv')
test_embs = torch.Tensor(np.load('test_fingerprints/fp_circ_test.npy'))

In [17]:
class Docking_Prediction(nn.Module):
    def __init__(self, layer_dims):
        super().__init__()
        
        self.seq = []
        for i in range(len(layer_dims)-1):
            self.seq.append(nn.Linear(layer_dims[i], layer_dims[i+1]))
#            self.seq.append(nn.Dropout(dropout_rate, inplace=True))
            if i!=len(layer_dims)-2:
#                self.seq.append(nn.BatchNorm1d(layer_dims[i+1]))
                self.seq.append(nn.LeakyReLU())   
        self.sequential = nn.Sequential(*self.seq)
    
    def forward(self, x):
        return self.sequential(x)    

In [33]:
for i, col in enumerate(test.columns[1:]):
    layer_dims = NN_arch[i]
    model = Docking_Prediction(layer_dims)
    model.load_state_dict(torch.load('BestNN_circ/bestNN_circ_col{}'.format(i)))#load the best model
    preds = model(test_embs).detach().numpy()
    print(i, preds.std())
    test[col] = preds

0 1.0529622
1 1.2074009
2 1.2045599
3 0.679616
4 0.68399405
5 0.9718694
6 0.4763172
7 0.47155827
8 0.8732726
9 1.0249151
10 0.9919686
11 0.9319991
12 0.8627356
13 0.66359633
14 0.67823106
15 0.8811654
16 0.9275786
17 1.0742172


In [35]:
test.to_csv('test_results.csv')